# Install

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install gdown

import torch
import gdown
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer
from trl import SFTTrainer
from sklearn.model_selection import train_test_split


## Connect drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load data

In [ ]:
import pandas as pd

df = pd.read_csv("/content/dataset.csv")

df

## Split data

In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df.head, test_size=0.1, random_state=42)

dataset = {
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(valid_df),
}

print(f"Training size: {len(dataset['train'])}, Validation size: {len(dataset['validation'])}")

## Load model

In [ ]:
import torch
from unsloth import FastLanguageModel
from huggingface_hub import login

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "vilm/vinallama-2.7b-chat",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",  # Attention
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 2902,
    use_rslora = False,
    loftq_config = None,
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

## Prompt

In [ ]:
import textwrap

def format_poem(example):
    theme = example["title"]
    content = example["content"].strip()
    text = f"""<|im_start|>system
Bạn là một AI thi sĩ chuyên sáng tác thơ lục bát bằng tiếng Việt.
Hãy thể hiện cảm xúc tinh tế, sử dụng ngôn từ đẹp, và tuân thủ nghiêm ngặt các quy tắc sau:
- Bài thơ có nhiều cặp dòng: một dòng 6 chữ, sau đó là một dòng 8 chữ.
- Mỗi dòng phải xuống dòng rõ ràng.
- Dòng 6 chữ có đúng 6 từ, dòng 8 chữ có đúng 8 từ.
- Không được viết sai nhịp, không viết quá, viết thiếu số chữ.
- Vần luật phải tự nhiên, đúng phong cách thơ ca truyền thống Việt Nam.
<|im_end|>
<|im_start|>user
Hãy sáng tác một bài thơ lục bát về chủ đề '{theme}'<|im_end|>
<|im_start|>assistant
{content}<|im_end|>
""".strip()

    return {"text": text}

# update dataset
dataset["train"] = dataset["train"].map(
    format_poem,
    remove_columns=dataset["train"].column_names
)

dataset["validation"] = dataset["validation"].map(
    format_poem,
    remove_columns=dataset["validation"].column_names
)

print(dataset["train"][0]["text"])


## Tokenizer

In [ ]:
def tokenize_function(example):
    tokenized = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length
    )

    #label
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


In [ ]:
tokenized_datasets = {
    "train": dataset["train"].map(
        tokenize_function,
        batched=True,
        remove_columns=["text"]
    ),
    "validation": dataset["validation"].map(
        tokenize_function,
        batched=True,
        remove_columns=["text"]
    )
}


## Data collator

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    return_tensors="pt"
)


In [ ]:
batch = [tokenized_datasets["train"][i] for i in range(2)]

collated_batch = data_collator(batch)

print(collated_batch.keys())  # ['input_ids', 'attention_mask', 'labels']
print("Input IDs shape:", collated_batch["input_ids"].shape)    # torch.Size([2, 2048])
print("Labels shape:", collated_batch["labels"].shape)          # torch.Size([2, 2048])

## Train

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/vinallama-checkpoints",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    max_steps=10000,
    warmup_steps=100,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    report_to=[],

    logging_strategy="steps",
    logging_steps=50,

    eval_strategy="steps",
    eval_steps=200,
    do_eval=True,

    save_strategy="steps",
    save_steps=200,
    save_total_limit=3
)


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    max_seq_length=max_seq_length,
    data_collator=data_collator,
    dataset_num_proc=2,
    packing=False,
    args=training_args
)


In [ ]:
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/vinallama-checkpoints/checkpoint-1320")